1. Disneyland Calendar Data Scrape

In [1]:
#Import Dependencies
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import calendar
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

In [2]:
#Set Preferences
myoptions = webdriver.ChromeOptions()
prefs = {
  "translate_whitelists": {"ja":"en"},
  "translate":{"enabled":"true"}
}
myoptions.add_experimental_option("prefs", prefs)
d = webdriver.Chrome('C:\Windows/chromedriver', options=myoptions)

#Define URL
url = "http://www15.plala.or.jp/gcap/disney/"
d.get(url)
d.maximize_window()

In [6]:
#Create Lists to Place Scraped Data
months = []
for monthnum in range(3):

    #Scrape Calendar Data
    elem = d.find_element_by_class_name("FXTABLE") 
    data = elem.text

    i = 1
    new_data = []
    temp = []
    for e in (data.split("\n")[14:]):
        temp.append(e)
        if i%4==0:
            new_data.append(temp)
            temp = []
        i = i+1
    #Place Scraped Data into DataFrame
    months.append(pd.DataFrame(new_data, columns=["Date", "Crowd Size", "Temperature", "Opening Hours"]))
    #Go to Previous Month
    element = WebDriverWait(d, 15).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="hyo"]/table/tbody/tr[2]/td[1]/font/font/input'))
    )
    previous = d.find_element_by_xpath('//*[@id="hyo"]/table/tbody/tr[2]/td[1]/font/font/input')
    ActionChains(d).click(previous).perform()
    element = WebDriverWait(d, 15).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="hyo"]/table/tbody/tr[2]/td[1]/font/font/input'))
    )
print(months)

[            Date Crowd Size Temperature Opening Hours
0             27         16      12 | 5          9-21
1             28         16      12 | 5          9-21
2             29         16      12 | 6          9-21
3             30         16      10 | 4          9-21
4             31         16       6 | 0          9-21
5            1/1         16       9 | 1          9-21
6              2         16      10 | 0          9-21
7              3         16       8 | 0          9-21
8           Four         16      11 | 1          9-21
9           Five         16       9 | 2          9-21
10             6         16       6 | 4         10-21
11             7         16      12 | 3         10-21
12             8         16       6 | 0          9-20
13             9         16       7 | 0          9-20
14           Ten         16      7 | -2          9-20
15            11         16      6 | -2          9-20
16            12         16       4 | 1          9-19
17            13         16

In [7]:
#Place Scraped Data into DataFrame
#disneyland_scrape = pd.concat(months)

In [8]:
#Save DataFrame as CSV
#disneyland_scrape.to_csv(r'C:\Users\Mickey\Desktop\Final_Project\disneyland_scrape.csv', index = False)